In [10]:
import bw2data as bd
import bw2calc as bc
import bw_processing as bwp

In [2]:
bd.projects.set_current('GSA for archetypes')

In [3]:
bd.databases

Databases dictionary with 3 object(s):
	biosphere3
	ecoinvent 3.8 cutoff
	swiss consumption 1.0

# Local sensitivity anaysis

We want to be able to change one exchange by a constant factor and test the effect on LCA scores.

The most efficient way to do this is with an `Interface`, so we don't have to generate many near-identical columns with only one element changing.

Our interface will be based on the `ecoinvent` data package, so let's start with that.

In [5]:
ei = bd.Database('ecoinvent 3.8 cutoff').datapackage()
ei

datapackages can have multiple resources; we only need the technosphere (for now)

In [8]:
ei = ei.filter_by_attribute('matrix', 'technosphere_matrix')

In [9]:
[x['name'] for x in ei.resources]

['ecoinvent_3.8_cutoff_technosphere_matrix.indices',
 'ecoinvent_3.8_cutoff_technosphere_matrix.data',
 'ecoinvent_3.8_cutoff_technosphere_matrix.distributions',
 'ecoinvent_3.8_cutoff_technosphere_matrix.flip']

Construct an `Interface` that iteratively changes every exchange with uncertainty by `factor`.

In [57]:
class LocalSAInterface:
    def __init__(self, indices, data, distributions, factor=10):
        self.indices = indices
        self.data = data
        self.distributions = distributions
        self.factor = factor

        assert self.indices.shape[0] == self.data.shape[0] == self.distributions.shape[0]
        
        self.size = len(self.indices)
        self.index = None  # To indicate we haven't consumed first value yet
        
    def __next__(self):
        if self.index is None:
            self.index = 0
        else:
            self.index += 1
            
        while self.distributions[self.index]['uncertainty_type'] < 2: # 0 and 1 are no and unknown uncertainty                             
            self.index += 1
            if self.index >= self.size:
                raise StopIteration

        data = self.data.copy()
        data[self.index] *= self.factor
        return data
    
    @property
    def coordinates(self):
        return self.indices[self.index]

In [58]:
tech_interface = LocalSAInterface(
    ei.get_resource('ecoinvent_3.8_cutoff_technosphere_matrix.indices')[0],
    ei.get_resource('ecoinvent_3.8_cutoff_technosphere_matrix.data')[0],
    ei.get_resource('ecoinvent_3.8_cutoff_technosphere_matrix.distributions')[0],
    10
)

The interface needs to be in a datapackage; we create one in memory.

In [59]:
dp = bwp.create_datapackage()
dp.add_dynamic_vector(
    matrix = 'technosphere_matrix',
    interface = tech_interface,
    indices_array = ei.get_resource('ecoinvent_3.8_cutoff_technosphere_matrix.indices')[0],
    flip_array = ei.get_resource('ecoinvent_3.8_cutoff_technosphere_matrix.flip')[0],  # Only needed for technosphere
)

How do we use this datapackage? We can just append it to the list of datapackages. First, the setup for the static LCA:

In [60]:
fu = {bd.Database('ecoinvent 3.8 cutoff').random(): 1}

In [61]:
fu_mapped, packages, _ = bd.prepare_lca_inputs(demand=fu, remapping=False)  # Could also add LCIA method

In [62]:
lca = bc.LCA(demand=fu_mapped, data_objs=packages)

In [63]:
lca.lci()
lca.inventory.sum()

85.14556996360245

And now with our local SA interface:

In [64]:
lca2 = bc.LCA(demand=fu_mapped, data_objs=packages + [dp])

In [65]:
lca2.lci()

In [71]:
for x in range(10):
    next(lca2) # Will also do lci and lcia calculation as needed
    i, j = tech_interface.coordinates
    row, col = lca2.dicts.product[i], lca2.dicts.activity[j]
    print(
        lca.inventory.sum(), 
        tech_interface.index, 
        tech_interface.coordinates, row, col,
        lca2.technosphere_matrix[row, col], 
        lca.technosphere_matrix[row, col] 
    )
    # product = bd.get_activity(id=i)
    # activity = bd.get_activity(id=j)
    # print(product, activity)

85.14556996360245 49 (4433, 7360) 5 2932 -5.7573628425598145 -0.5757362842559814
85.14556996360245 51 (4433, 7532) 5 3104 -0.2459999918937683 -0.02459999918937683
85.14556996360245 52 (4433, 7549) 5 3121 -0.5625190734863281 -0.05625190958380699
85.14556996360245 53 (4433, 7587) 5 3159 -0.19999998807907104 -0.019999999552965164
85.14556996360245 54 (4433, 7831) 5 3403 -0.26381751894950867 -0.026381751522421837
85.14556996360245 55 (4433, 7983) 5 3555 -0.03199999779462814 -0.0031999999191612005
85.14556996360245 56 (4433, 8009) 5 3581 -0.29999998211860657 -0.029999999329447746
85.14556996360245 57 (4433, 8131) 5 3703 -0.2459999918937683 -0.02459999918937683
85.14556996360245 58 (4433, 8170) 5 3742 -0.9000000357627869 -0.09000000357627869
85.14556996360245 60 (4433, 8248) 5 3820 -0.4699999690055847 -0.04699999839067459


Actually calculating the local SA indices with a LCIA method is left as an exercise for the reader :)